In [24]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler

In [2]:
df = pd.read_csv('./boston/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 15 columns):
ID         333 non-null int64
crim       333 non-null float64
zn         333 non-null float64
indus      333 non-null float64
chas       333 non-null int64
nox        333 non-null float64
rm         333 non-null float64
age        333 non-null float64
dis        333 non-null float64
rad        333 non-null int64
tax        333 non-null int64
ptratio    333 non-null float64
black      333 non-null float64
lstat      333 non-null float64
medv       333 non-null float64
dtypes: float64(11), int64(4)
memory usage: 39.1 KB


In [39]:
X = df[['nox', 'rm', 'chas', 'dis', 'ptratio', 'lstat', 'rad']].values
y = df['medv'].values

In [40]:
print(X.shape)

(333, 7)


In [17]:
print(X)

[[6.575  0.538 ]
 [6.421  0.469 ]
 [6.998  0.458 ]
 [7.147  0.458 ]
 [6.012  0.524 ]
 [6.377  0.524 ]
 [6.009  0.524 ]
 [5.889  0.524 ]
 [5.949  0.538 ]
 [6.096  0.538 ]
 [5.834  0.538 ]
 [5.935  0.538 ]
 [5.456  0.538 ]
 [5.57   0.538 ]
 [5.965  0.538 ]
 [6.142  0.538 ]
 [5.813  0.538 ]
 [6.047  0.538 ]
 [5.713  0.538 ]
 [6.072  0.538 ]
 [6.096  0.538 ]
 [5.966  0.499 ]
 [6.595  0.428 ]
 [7.024  0.428 ]
 [6.169  0.448 ]
 [6.211  0.448 ]
 [6.069  0.448 ]
 [5.682  0.448 ]
 [5.786  0.448 ]
 [6.03   0.448 ]
 [5.602  0.448 ]
 [5.963  0.439 ]
 [6.115  0.439 ]
 [5.998  0.439 ]
 [5.888  0.41  ]
 [7.249  0.403 ]
 [6.383  0.41  ]
 [6.816  0.411 ]
 [6.145  0.453 ]
 [5.741  0.453 ]
 [5.966  0.453 ]
 [6.762  0.453 ]
 [7.104  0.4161]
 [6.29   0.398 ]
 [5.787  0.398 ]
 [5.878  0.409 ]
 [5.594  0.409 ]
 [6.417  0.413 ]
 [6.065  0.413 ]
 [6.245  0.413 ]
 [6.273  0.437 ]
 [6.286  0.437 ]
 [6.279  0.437 ]
 [6.14   0.437 ]
 [6.727  0.426 ]
 [6.619  0.426 ]
 [6.167  0.426 ]
 [6.389  0.449 ]
 [6.63   0.449

In [41]:
scaler = MinMaxScaler()
scaled_X = scaler.fit_transform(X)

In [26]:
print(scaled_X)

[[0.58365608 0.31481481]
 [0.55383424 0.17283951]
 [0.66556933 0.15020576]
 [0.69442293 0.15020576]
 [0.47463207 0.28600823]
 [0.54531371 0.28600823]
 [0.47405112 0.28600823]
 [0.45081332 0.28600823]
 [0.46243222 0.31481481]
 [0.49089853 0.31481481]
 [0.44016266 0.31481481]
 [0.45972115 0.31481481]
 [0.36696359 0.31481481]
 [0.3890395  0.31481481]
 [0.4655306  0.31481481]
 [0.49980635 0.31481481]
 [0.43609605 0.31481481]
 [0.48140976 0.31481481]
 [0.41673122 0.31481481]
 [0.48625097 0.31481481]
 [0.49089853 0.31481481]
 [0.46572424 0.2345679 ]
 [0.58752905 0.08847737]
 [0.67060418 0.08847737]
 [0.50503486 0.12962963]
 [0.51316809 0.12962963]
 [0.48567002 0.12962963]
 [0.41072812 0.12962963]
 [0.43086754 0.12962963]
 [0.47811774 0.12962963]
 [0.39523625 0.12962963]
 [0.4651433  0.11111111]
 [0.49457785 0.11111111]
 [0.47192099 0.11111111]
 [0.45061967 0.05144033]
 [0.71417506 0.03703704]
 [0.5464756  0.05144033]
 [0.63032533 0.05349794]
 [0.5003873  0.1399177 ]
 [0.42215337 0.1399177 ]


In [42]:
poly = PolynomialFeatures(2)

In [43]:
new_X = poly.fit_transform(scaled_X)

In [44]:
print(new_X.shape)

(333, 36)


In [38]:
print(new_X)

[[1.         0.58365608 0.31481481 ... 0.10724306 0.0578452  0.03120078]
 [1.         0.55383424 0.17283951 ... 0.05301547 0.01654496 0.00516332]
 [1.         0.66556933 0.15020576 ... 0.06653853 0.01501642 0.00338891]
 ...
 [1.         0.49554609 0.38683128 ... 0.09499258 0.07415274 0.05788483]
 [1.         0.66130906 0.38683128 ... 0.1691728  0.09895725 0.05788483]
 [1.         0.47811774 0.38683128 ... 0.0884283  0.07154479 0.05788483]]


In [30]:
X_train, X_test, y_train, y_test = train_test_split(new_X, y, test_size=0.1, random_state=40)

In [31]:
lr = LinearRegression()
lr.fit(X_train, y_train)
print('Score: {}'.format(lr.score(X_test, y_test)))

Score: 0.8215496485848502
